# Problem Definition

The Robot Weightlifting World Championship’s final round is about to begin! Three robots, seeded 1, 2, and 3, remain in contention. They take turns from the 3rd seed to the 1st seed publicly declaring exactly how much weight (any nonnegative real number) they will attempt to lift, and no robot can choose exactly the same amount as a previous robot. Once the three weights have been announced, the robots attempt their lifts, and the robot that successfully lifts the most weight is the winner. If all robots fail, they just repeat the same lift amounts until at least one succeeds.

Assume the following:

1) all the robots have the same probability p(w) of successfully lifting a given weight w;

2) p(w) is exactly known by all competitors, continuous, strictly decreasing as the w increases, p(0) = 1, and p(w) -> 0 as w -> infinity; and

3) all competitors want to maximize their chance of winning the RWWC.

If w is the amount of weight the 3rd seed should request, find p(w). Give your answer to an accuracy of six decimal places.

# A wrong solution.... Well, I didn't read the problem definition properly

This is an incorrect solution. I didn't realize that every competitor wants to maximize their chance of winning the RWWC and have gone for another approach instead. My solution is maximizing the chance of player 3 winning, assuming that the two other robots would take the choice that hurts player 3 the most. Basically, this is the approach to take if I don't know what selection criteria the other robots have. I.e. they might chose their p randomly or they might want to cause as much damage as possible.

In [1]:
from random import random

def simulate_win_probabilities(chosen_probabilities, num_simulations = 10000):
    player_wins = [0 for i in chosen_probabilities]
    for i in range(num_simulations):
        winner = -1
        lowest_prob = 1
        while winner == -1:
            for player, prob in enumerate(chosen_probabilities):
                r = random()
                if r <= prob:
                    if prob < lowest_prob:
                        lowest_prob = prob
                        winner = player
        player_wins[winner] += 1
    return [player_wins[p]/num_simulations for p in range(len(chosen_probabilities))]

In [2]:
simulate_win_probabilities([0.8, 0.4], 1000)

[0.561, 0.439]

In [3]:
def calculate_win_probabilities(p1, p2, rec = 2000):
    if rec == 0:
        return 1
    else:
        if p1 > p2:
            return p1*(1-p2)+(1-p1)*(1-p2)*calculate_win_probabilities(p1, p2, rec-1)
        if p1 < p2:
            return p1 + (1-p1)*(1-p2)*calculate_win_probabilities(p1, p2, rec-1)

In [4]:
for rec in range(40):
    print('%4i: %1.10f'%(rec, calculate_win_probabilities(0.9, 0.4, rec)))

   0: 1.0000000000
   1: 0.6000000000
   2: 0.5760000000
   3: 0.5745600000
   4: 0.5744736000
   5: 0.5744684160
   6: 0.5744681050
   7: 0.5744680863
   8: 0.5744680852
   9: 0.5744680851
  10: 0.5744680851
  11: 0.5744680851
  12: 0.5744680851
  13: 0.5744680851
  14: 0.5744680851
  15: 0.5744680851
  16: 0.5744680851
  17: 0.5744680851
  18: 0.5744680851
  19: 0.5744680851
  20: 0.5744680851
  21: 0.5744680851
  22: 0.5744680851
  23: 0.5744680851
  24: 0.5744680851
  25: 0.5744680851
  26: 0.5744680851
  27: 0.5744680851
  28: 0.5744680851
  29: 0.5744680851
  30: 0.5744680851
  31: 0.5744680851
  32: 0.5744680851
  33: 0.5744680851
  34: 0.5744680851
  35: 0.5744680851
  36: 0.5744680851
  37: 0.5744680851
  38: 0.5744680851
  39: 0.5744680851


In [5]:
def calculate_win_probabilities3(p1, p2, p3, rec = 2000):
    if rec == 0:
        return 1
    else:
        if p1 > p2 and p1 > p3:
            return p1*(1-p2)*(1-p3) + (1-p1)*(1-p2)*(1-p3)*calculate_win_probabilities3(p1, p2, p3, rec-1)
        if p2 > p1 and p1 > p3:
            return p1*(1-p3) + (1-p1)*(1-p2)*(1-p3)*calculate_win_probabilities3(p1, p2, p3, rec-1)
        if p3 > p1 and p1 > p2:
            return p1*(1-p2) + (1-p1)*(1-p2)*(1-p3)*calculate_win_probabilities3(p1, p2, p3, rec-1)
        if p2 > p1 and p3 > p1:
            return p1 + (1-p1)*(1-p2)*(1-p3)*calculate_win_probabilities3(p1, p2, p3, rec-1)

In [6]:
simulate_win_probabilities([0.8, 0.4, 0.1], 100000)

[0.48215, 0.40654, 0.11131]

In [7]:
calculate_win_probabilities3(0.8, 0.4, 0.1)

0.48430493273542596

In [8]:
calculate_win_probabilities3(0.4, 0.8, 0.1)

0.4035874439461884

In [9]:
calculate_win_probabilities3(0.1, 0.8, 0.4)

0.11210762331838565

In [10]:
calculate_win_probabilities3(0.8, 0.4, 0.1, 20)

0.48430493273542596

In [11]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

In [12]:
def test_through_p1_range(start, end, increments, rec_steps = 100, p2_ranges = [[0, 1]], p3_ranges = [[0, 1]]):
    best_so_far = 0
    for p1 in frange(start, end, increments):
        worst_p = 1
        enemy_p2 = 0
        enemy_p3 = 0
        for p2_range in p2_ranges:
            for p2 in frange(p2_range[0], p2_range[1], increments):
                for p3_range in p3_ranges:
                    for p3 in frange(p3_range[0], p3_range[1], increments):
                        if p1 != p2 and p2 < p3 and p1 != p3:
                            p = calculate_win_probabilities3(p1, p2, p3, rec_steps)
                            if p < worst_p:
                                worst_p = p
                                enemy_p2 = p2
                                enemy_p3 = p3
        if worst_p > best_so_far:
            best_so_far = worst_p
        best = '*' if best_so_far == worst_p else ''
        print('%1.10f : %1.10f (enemy_p2: %1.10f, enemy_p3: %1.10f) %s'%(p1, worst_p, enemy_p2, enemy_p3, best))

In [13]:
test_through_p1_range(0.01, 1, 0.01, 50)

0.0100000000 : 0.0100019804 (enemy_p2: 0.9800000000, enemy_p3: 0.9900000000) *
0.0200000000 : 0.0199939816 (enemy_p2: 0.0100000000, enemy_p3: 0.9900000000) *
0.0300000000 : 0.0296821586 (enemy_p2: 0.0200000000, enemy_p3: 0.9900000000) *
0.0400000000 : 0.0391647017 (enemy_p2: 0.0300000000, enemy_p3: 0.9900000000) *
0.0500000000 : 0.0484417891 (enemy_p2: 0.0400000000, enemy_p3: 0.9900000000) *
0.0600000000 : 0.0575135964 (enemy_p2: 0.0500000000, enemy_p3: 0.9900000000) *
0.0700000000 : 0.0663802966 (enemy_p2: 0.0600000000, enemy_p3: 0.9900000000) *
0.0800000000 : 0.0750420599 (enemy_p2: 0.0700000000, enemy_p3: 0.9900000000) *
0.0900000000 : 0.0834990541 (enemy_p2: 0.0800000000, enemy_p3: 0.9900000000) *
0.1000000000 : 0.0917514443 (enemy_p2: 0.0900000000, enemy_p3: 0.9900000000) *
0.1100000000 : 0.0997993931 (enemy_p2: 0.1000000000, enemy_p3: 0.9900000000) *
0.1200000000 : 0.1076430604 (enemy_p2: 0.1100000000, enemy_p3: 0.9900000000) *
0.1300000000 : 0.1152826036 (enemy_p2: 0.1200000000,

In [14]:
test_through_p1_range(0.317, 0.324, 0.001, 100)

0.3170000000 : 0.2169293433 (enemy_p2: 0.3160000000, enemy_p3: 0.9990000000) *
0.3180000000 : 0.2172952174 (enemy_p2: 0.3170000000, enemy_p3: 0.9990000000) *
0.3190000000 : 0.2176261511 (enemy_p2: 0.3170000000, enemy_p3: 0.3180000000) *
0.3200000000 : 0.2172258202 (enemy_p2: 0.3180000000, enemy_p3: 0.3190000000) 
0.3210000000 : 0.2168253661 (enemy_p2: 0.3190000000, enemy_p3: 0.3200000000) 
0.3220000000 : 0.2164247892 (enemy_p2: 0.3200000000, enemy_p3: 0.3210000000) 
0.3230000000 : 0.2160240900 (enemy_p2: 0.3210000000, enemy_p3: 0.3220000000) 


In [15]:
test_through_p1_range(0.3174, 0.318, 0.0001, 100, [[0.317,0.32], [0.98,1]], [[0.317,0.32], [0.98,1]])

0.3174000000 : 0.2166572400 (enemy_p2: 0.3174000000, enemy_p3: 1.0000000000) *
0.3175000000 : 0.2166937500 (enemy_p2: 0.3175000000, enemy_p3: 1.0000000000) *
0.3176000000 : 0.2167302400 (enemy_p2: 0.3176000000, enemy_p3: 1.0000000000) *
0.3177000000 : 0.2167667100 (enemy_p2: 0.3177000000, enemy_p3: 1.0000000000) *
0.3178000000 : 0.2167524152 (enemy_p2: 0.3177000000, enemy_p3: 0.3178000000) 
0.3179000000 : 0.2167127300 (enemy_p2: 0.3178000000, enemy_p3: 0.3179000000) 
0.3180000000 : 0.2166730435 (enemy_p2: 0.3179000000, enemy_p3: 0.3180000000) 


In [16]:
test_through_p1_range(0.3174, 0.318, 0.00005, 100, [[0.317,0.32], [0.98,1]], [[0.317,0.32], [0.98,1]])

0.3174000000 : 0.2166572400 (enemy_p2: 0.3174000000, enemy_p3: 1.0000000000) *
0.3174500000 : 0.2166754975 (enemy_p2: 0.3174500000, enemy_p3: 1.0000000000) *
0.3175000000 : 0.2166937500 (enemy_p2: 0.3175000000, enemy_p3: 1.0000000000) *
0.3175500000 : 0.2167119975 (enemy_p2: 0.3175500000, enemy_p3: 1.0000000000) *
0.3176000000 : 0.2167302400 (enemy_p2: 0.3176000000, enemy_p3: 1.0000000000) *
0.3176500000 : 0.2167484775 (enemy_p2: 0.3176500000, enemy_p3: 1.0000000000) *
0.3177000000 : 0.2167667100 (enemy_p2: 0.3177000000, enemy_p3: 1.0000000000) *
0.3177500000 : 0.2167489807 (enemy_p2: 0.3177000000, enemy_p3: 0.3177500000) 
0.3178000000 : 0.2167291414 (enemy_p2: 0.3177500000, enemy_p3: 0.3178000000) 
0.3178500000 : 0.2167093018 (enemy_p2: 0.3178000000, enemy_p3: 0.3178500000) 
0.3179000000 : 0.2166894619 (enemy_p2: 0.3178500000, enemy_p3: 0.3179000000) 
0.3179500000 : 0.2166696216 (enemy_p2: 0.3179000000, enemy_p3: 0.3179500000) 
0.3180000000 : 0.2166497809 (enemy_p2: 0.3179500000, enem

In [17]:
test_through_p1_range(0.3176, 0.31771, 0.00001, 100, [[0.3172,0.318], [0.99,1]], [[0.3172,0.318], [0.99,1]])

0.3176000000 : 0.2167334160 (enemy_p2: 0.3175900000, enemy_p3: 1.0000000000) *
0.3176100000 : 0.2167370640 (enemy_p2: 0.3176000000, enemy_p3: 1.0000000000) *
0.3176200000 : 0.2167407118 (enemy_p2: 0.3176100000, enemy_p3: 1.0000000000) *
0.3176300000 : 0.2167443594 (enemy_p2: 0.3176200000, enemy_p3: 1.0000000000) *
0.3176400000 : 0.2167480068 (enemy_p2: 0.3176300000, enemy_p3: 1.0000000000) *
0.3176500000 : 0.2167516540 (enemy_p2: 0.3176400000, enemy_p3: 1.0000000000) *
0.3176600000 : 0.2167553010 (enemy_p2: 0.3176500000, enemy_p3: 1.0000000000) *
0.3176700000 : 0.2167589478 (enemy_p2: 0.3176600000, enemy_p3: 1.0000000000) *
0.3176800000 : 0.2167625944 (enemy_p2: 0.3176700000, enemy_p3: 1.0000000000) *
0.3176900000 : 0.2167634759 (enemy_p2: 0.3176700000, enemy_p3: 0.3176800000) *
0.3177000000 : 0.2167595084 (enemy_p2: 0.3176800000, enemy_p3: 0.3176900000) 


In [18]:
test_through_p1_range(0.31765, 0.31775, 0.00001, 100, [[0.3176,0.3178], [0.9999,1]], [[0.3176,0.3178], [0.9999,1]])

0.3176500000 : 0.2167516540 (enemy_p2: 0.3176400000, enemy_p3: 1.0000000000) *
0.3176600000 : 0.2167553010 (enemy_p2: 0.3176500000, enemy_p3: 1.0000000000) *
0.3176700000 : 0.2167589478 (enemy_p2: 0.3176600000, enemy_p3: 1.0000000000) *
0.3176800000 : 0.2167625944 (enemy_p2: 0.3176700000, enemy_p3: 1.0000000000) *
0.3176900000 : 0.2167634759 (enemy_p2: 0.3176700000, enemy_p3: 0.3176800000) *
0.3177000000 : 0.2167595084 (enemy_p2: 0.3176800000, enemy_p3: 0.3176900000) 
0.3177100000 : 0.2167555408 (enemy_p2: 0.3176900000, enemy_p3: 0.3177000000) 
0.3177200000 : 0.2167515733 (enemy_p2: 0.3177000000, enemy_p3: 0.3177100000) 
0.3177300000 : 0.2167476058 (enemy_p2: 0.3177100000, enemy_p3: 0.3177200000) 
0.3177400000 : 0.2167436382 (enemy_p2: 0.3177200000, enemy_p3: 0.3177300000) 


In [19]:
test_through_p1_range(0.31766, 0.317681, 0.000001, 100, [[0.31762,0.31775], [0.9999,1]], [[0.31762,0.31775], [0.9999,1]])

0.3176600000 : 0.2167522253 (enemy_p2: 0.3176600000, enemy_p3: 0.9999990000) *
0.3176610000 : 0.2167525900 (enemy_p2: 0.3176610000, enemy_p3: 0.9999990000) *
0.3176620000 : 0.2167529547 (enemy_p2: 0.3176620000, enemy_p3: 0.9999990000) *
0.3176630000 : 0.2167533193 (enemy_p2: 0.3176630000, enemy_p3: 0.9999990000) *
0.3176640000 : 0.2167536840 (enemy_p2: 0.3176640000, enemy_p3: 0.9999990000) *
0.3176650000 : 0.2167540487 (enemy_p2: 0.3176650000, enemy_p3: 0.9999990000) *
0.3176660000 : 0.2167544134 (enemy_p2: 0.3176660000, enemy_p3: 0.9999990000) *
0.3176670000 : 0.2167547780 (enemy_p2: 0.3176670000, enemy_p3: 0.9999990000) *
0.3176680000 : 0.2167551427 (enemy_p2: 0.3176680000, enemy_p3: 0.9999990000) *
0.3176690000 : 0.2167555074 (enemy_p2: 0.3176690000, enemy_p3: 0.9999990000) *
0.3176700000 : 0.2167558720 (enemy_p2: 0.3176700000, enemy_p3: 0.9999990000) *
0.3176710000 : 0.2167562367 (enemy_p2: 0.3176710000, enemy_p3: 0.9999990000) *
0.3176720000 : 0.2167566013 (enemy_p2: 0.3176720000,

In [20]:
test_through_p1_range(0.317669, 0.3176745, 0.0000005, 100, [[0.317661,0.317681], [0.9999,1]], [[0.317661,0.317681], [0.9999,1]])

0.3176690000 : 0.2167555653 (enemy_p2: 0.3176685000, enemy_p3: 1.0000000000) *
0.3176695000 : 0.2167557476 (enemy_p2: 0.3176690000, enemy_p3: 1.0000000000) *
0.3176700000 : 0.2167559299 (enemy_p2: 0.3176695000, enemy_p3: 1.0000000000) *
0.3176705000 : 0.2167561123 (enemy_p2: 0.3176700000, enemy_p3: 1.0000000000) *
0.3176710000 : 0.2167562946 (enemy_p2: 0.3176705000, enemy_p3: 1.0000000000) *
0.3176715000 : 0.2167564769 (enemy_p2: 0.3176710000, enemy_p3: 1.0000000000) *
0.3176720000 : 0.2167566593 (enemy_p2: 0.3176715000, enemy_p3: 1.0000000000) *
0.3176725000 : 0.2167568416 (enemy_p2: 0.3176720000, enemy_p3: 1.0000000000) *
0.3176730000 : 0.2167569514 (enemy_p2: 0.3176720000, enemy_p3: 0.3176725000) *
0.3176735000 : 0.2167567531 (enemy_p2: 0.3176725000, enemy_p3: 0.3176730000) 
0.3176740000 : 0.2167565547 (enemy_p2: 0.3176730000, enemy_p3: 0.3176735000) 


In [21]:
test_through_p1_range(0.317672, 0.3176726, 0.0000001, 100, [[0.317668,0.3176749], [0.9999,1]], [[0.317668,0.3176749], [0.9999,1]])

0.3176720000 : 0.2167565322 (enemy_p2: 0.3176719000, enemy_p3: 1.0000000000) *
0.3176721000 : 0.2167565686 (enemy_p2: 0.3176720000, enemy_p3: 1.0000000000) *
0.3176722000 : 0.2167566051 (enemy_p2: 0.3176721000, enemy_p3: 1.0000000000) *
0.3176723000 : 0.2167566416 (enemy_p2: 0.3176722000, enemy_p3: 1.0000000000) *
0.3176724000 : 0.2167566308 (enemy_p2: 0.3176722000, enemy_p3: 0.3176723000) 
0.3176725000 : 0.2167565911 (enemy_p2: 0.3176723000, enemy_p3: 0.3176724000) 


In [22]:
test_through_p1_range(0.3176721, 0.3176726, 0.00000005, 100, [[0.3176719,0.3176729], [0.999999,1]], [[0.3176719,0.3176729], [0.999999,1]])

0.3176721000 : 0.2167565419 (enemy_p2: 0.3176721000, enemy_p3: 0.9999999500) *
0.3176721500 : 0.2167565602 (enemy_p2: 0.3176721500, enemy_p3: 0.9999999500) *
0.3176722000 : 0.2167565784 (enemy_p2: 0.3176722000, enemy_p3: 0.9999999500) *
0.3176722500 : 0.2167565739 (enemy_p2: 0.3176722000, enemy_p3: 0.3176722500) 
0.3176723000 : 0.2167565540 (enemy_p2: 0.3176722500, enemy_p3: 0.3176723000) 
0.3176723500 : 0.2167565342 (enemy_p2: 0.3176723000, enemy_p3: 0.3176723500) 
0.3176724000 : 0.2167565144 (enemy_p2: 0.3176723500, enemy_p3: 0.3176724000) 
0.3176724500 : 0.2167564945 (enemy_p2: 0.3176724000, enemy_p3: 0.3176724500) 
0.3176725000 : 0.2167564747 (enemy_p2: 0.3176724500, enemy_p3: 0.3176725000) 
0.3176725500 : 0.2167564549 (enemy_p2: 0.3176725000, enemy_p3: 0.3176725500) 
0.3176726000 : 0.2167564350 (enemy_p2: 0.3176725500, enemy_p3: 0.3176726000) 


In [23]:
test_through_p1_range(0.3176721, 0.31767227, 0.00000001, 100, [[0.3176719,0.3176729], [0.999999,1]], [[0.3176719,0.3176729], [0.999999,1]])

0.3176721000 : 0.2167565379 (enemy_p2: 0.3176721000, enemy_p3: 0.9999999900) *
0.3176721100 : 0.2167565415 (enemy_p2: 0.3176721100, enemy_p3: 0.9999999900) *
0.3176721200 : 0.2167565452 (enemy_p2: 0.3176721200, enemy_p3: 0.9999999900) *
0.3176721300 : 0.2167565488 (enemy_p2: 0.3176721300, enemy_p3: 0.9999999900) *
0.3176721400 : 0.2167565525 (enemy_p2: 0.3176721400, enemy_p3: 0.9999999900) *
0.3176721500 : 0.2167565561 (enemy_p2: 0.3176721500, enemy_p3: 0.9999999900) *
0.3176721600 : 0.2167565598 (enemy_p2: 0.3176721600, enemy_p3: 0.9999999900) *
0.3176721700 : 0.2167565634 (enemy_p2: 0.3176721700, enemy_p3: 0.9999999900) *
0.3176721800 : 0.2167565671 (enemy_p2: 0.3176721800, enemy_p3: 0.9999999900) *
0.3176721900 : 0.2167565707 (enemy_p2: 0.3176721900, enemy_p3: 0.9999999900) *
0.3176722000 : 0.2167565744 (enemy_p2: 0.3176722000, enemy_p3: 0.9999999900) *
0.3176722100 : 0.2167565711 (enemy_p2: 0.3176722000, enemy_p3: 0.3176722100) 
0.3176722200 : 0.2167565672 (enemy_p2: 0.3176722100, 

In [24]:
test_through_p1_range(0.31767217, 0.31767223, 0.000000005, 100, [[0.31767214,0.31767226], [0.999999,1]], [[0.31767214,0.31767226], [0.999999,1]])

0.3176721700 : 0.2167565640 (enemy_p2: 0.3176721650, enemy_p3: 1.0000000000) *
0.3176721750 : 0.2167565658 (enemy_p2: 0.3176721700, enemy_p3: 1.0000000000) *
0.3176721800 : 0.2167565676 (enemy_p2: 0.3176721750, enemy_p3: 1.0000000000) *
0.3176721850 : 0.2167565695 (enemy_p2: 0.3176721800, enemy_p3: 1.0000000000) *
0.3176721900 : 0.2167565713 (enemy_p2: 0.3176721850, enemy_p3: 1.0000000000) *
0.3176721950 : 0.2167565731 (enemy_p2: 0.3176721900, enemy_p3: 1.0000000000) *
0.3176722000 : 0.2167565749 (enemy_p2: 0.3176721950, enemy_p3: 1.0000000000) *
0.3176722050 : 0.2167565754 (enemy_p2: 0.3176721950, enemy_p3: 0.3176722000) *
0.3176722100 : 0.2167565734 (enemy_p2: 0.3176722000, enemy_p3: 0.3176722050) 
0.3176722150 : 0.2167565715 (enemy_p2: 0.3176722050, enemy_p3: 0.3176722100) 
0.3176722200 : 0.2167565695 (enemy_p2: 0.3176722100, enemy_p3: 0.3176722150) 
0.3176722250 : 0.2167565675 (enemy_p2: 0.3176722150, enemy_p3: 0.3176722200) 


In [25]:
test_through_p1_range(0.31767219, 0.317672201, 0.000000001, 100, [[0.31767214,0.31767226], [0.999999,1]], [[0.31767214,0.31767226], [0.999999,1]])

0.3176721900 : 0.2167565700 (enemy_p2: 0.3176721890, enemy_p3: 1.0000000000) *
0.3176721910 : 0.2167565704 (enemy_p2: 0.3176721900, enemy_p3: 1.0000000000) *
0.3176721920 : 0.2167565707 (enemy_p2: 0.3176721910, enemy_p3: 1.0000000000) *
0.3176721930 : 0.2167565711 (enemy_p2: 0.3176721920, enemy_p3: 1.0000000000) *
0.3176721940 : 0.2167565715 (enemy_p2: 0.3176721930, enemy_p3: 1.0000000000) *
0.3176721950 : 0.2167565718 (enemy_p2: 0.3176721940, enemy_p3: 1.0000000000) *
0.3176721960 : 0.2167565722 (enemy_p2: 0.3176721950, enemy_p3: 1.0000000000) *
0.3176721970 : 0.2167565726 (enemy_p2: 0.3176721960, enemy_p3: 1.0000000000) *
0.3176721980 : 0.2167565726 (enemy_p2: 0.3176721960, enemy_p3: 0.3176721970) *
0.3176721990 : 0.2167565722 (enemy_p2: 0.3176721970, enemy_p3: 0.3176721980) 
0.3176722000 : 0.2167565718 (enemy_p2: 0.3176721980, enemy_p3: 0.3176721990) 


In [26]:
test_through_p1_range(0.317672195, 0.3176721968, 0.000000001, 100, [[0.317672191,0.317672204], [0.9999999,1]], [[0.317672191,0.317672204], [0.9999999,1]])

0.3176721950 : 0.2167565718 (enemy_p2: 0.3176721940, enemy_p3: 1.0000000000) *
0.3176721960 : 0.2167565722 (enemy_p2: 0.3176721950, enemy_p3: 1.0000000000) *


In [27]:
simulate_win_probabilities([0.3176721964, 0.3176721962, 0.3176721963], 100000)

[0.21876, 0.46422, 0.31702]